In [37]:
import pandas as pd

def get_working_set(file_name):
    lds = set()
    sts = set()
    texs = set()
    count = pd.DataFrame(columns=['LD', 'ST', 'TEX', 'TEX_INST_COUNT','Total'])
    trace = open(file_name, 'r')
    lines = trace.readlines()
    lines.pop(0)
    tex_inst_count = 0

    for line in lines:
        if "LD" in line:
            splitted = line.split(" ")
            splitted.pop(0)
            for segment in splitted:
                if "0x" in segment:
                    # align to 128B
                    segment = int(segment, 16) & ~127
                    lds.add(segment)
        elif "ST" in line:
            splitted = line.split(" ")
            splitted.pop(0)
            for segment in splitted:
                if "0x" in segment:
                    # align to 128B
                    segment = int(segment, 16) & ~127
                    sts.add(segment)
        elif "TEX" in line:
            tex_inst_count += 1
            splitted = line.split(" ")
            splitted.pop(0)
            for segment in splitted:
                if "0x" in segment:
                    # align to 128B
                    segment = int(segment, 16) & ~127
                    texs.add(segment)
        
        elif "END_TB" in line:
            # print("Working set size: " + str(len(addrs)))
            count = count.append({'LD': len(lds), 'ST': len(sts), 'TEX': len(texs),
                                    'TEX_INST_COUNT': tex_inst_count, 
                                   'Total': len(lds) + len(sts) + len(texs)}, ignore_index=True)
            lds.clear()
            sts.clear()
            texs.clear()
            tex_inst_count = 0
            # break

    return count



In [118]:
sizes = get_working_set('/home/tgrogers-raid/a/pan251/accel-sim-framework/hw_run/traces/vulkan/render_passes_4k/NO_ARGS/traces/kernel-MESA_SHADER_FRAGMENT_func1_main_19.traceg')

print("avg:", sum(sizes['TEX'])/len(sizes['TEX']))
# print("min:", min(sizes))
# print("max:", max(sizes))
# # print("std:", (sum((x - sum(sizes)/len(sizes))**2 for x in sizes)/len(sizes))**0.5)
# print("median:", sorted(sizes)[len(sizes)//2])

sizes

avg: 4.518314513908014


,LD,ST,TEX,TEX_INST_COUNT,Total
0,11,23,3,2,37
1,11,24,2,2,37
2,11,56,1,2,68
3,11,35,2,2,48
4,11,37,4,2,52
...,...,...,...,...,...
3626,11,14,6,2,31
3627,11,18,5,2,34
3628,11,10,6,2,27
3629,11,15,4,2,30


In [ ]:
# print("max:", max(sizes))
# # print("std:", (sum((x - sum(sizes)/len(sizes))**2 for x in sizes)/len(sizes))**0.5)
# print("median:", sorted(sizes)[len(sizes)//2])

sizes

In [116]:
# plot
import plotly.express as px
import plotly.graph_objects as go

def plot(sizes):
    # fig = px.histogram(sizes[["LD", "ST", "TEX", "Total"]],
    #                 title = "Working Set Size Distribution", 
    #                 labels={'x':'Working Set Size'},
    #                 facet_row="variable",
    #                 nbins=100,
    #                     )
    for col in ["TEX"]:
        fig = go.Figure()
        fig.add_trace(go.Histogram(x=sizes[col], name=col, nbinsx=50))
        # chage width and height
        fig.update_layout(
            # autosize=False,
            width=800,
            height=300,
            margin=dict(
                l=10,
                r=10,
                b=10,
                t=50,
            ),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.01,
                xanchor="left",
                x=0
            ),
            autosize=True,
            xaxis_title="# of Cache Lines",
            yaxis_title="# of CTAs"
        )
        fig.update_yaxes(matches=None)
        fig.update_layout(
        xaxis=dict(
            titlefont=dict(size=25, color="black",family='sans-serif'),
            tickfont=dict(size=20, color="black",family='sans-serif'),
            autorange=True,
            gridcolor="gainsboro",
            showline=True,
            linecolor="black",
        ),
        yaxis=dict(
            titlefont=dict(size=25, color="black", family='sans-serif'),
            tickfont=dict(size=20, color="black", family='sans-serif'),
            autorange=True,
            gridcolor="gainsboro",
            showline=True,
            linecolor="black",
        ),
        title_font_family="sans-serif",
        title_font_size=25,
        margin=dict(l=20, r=10, t=20, b=20),
        plot_bgcolor="white",
        # xaxis=dict(gridcolor="gainsboro"),
        # yaxis=dict(gridcolor="gainsboro"),
    )
        fig.show()
        fig.write_image("/home/tgrogers-raid/a/pan251/graphics_accel_sim/Figures/{0}.pdf".format("tex_working_set"), format="pdf")

plot(sizes)

In [136]:
# list all files in a directory
import os
import threading

threads = []

stats = pd.DataFrame(columns=['mean'])

folder = "/home/tgrogers-raid/a/pan251/accel-sim-framework/hw_run/traces/vulkan/render_passes_2k/NO_ARGS/traces/"

means = []
def get_mean(filename_, means, index):
    sizes = get_working_set(folder + filename_)
    means[index] = sum(sizes['TEX'])/len(sizes['TEX'])
    print(filename_ + " mean:", means[index])

index = 0
for filename in os.listdir(folder):
    if filename.endswith(".traceg"):
        # if "MESA" in filename:
            # continue
        if "VERTEX" in filename:
            continue
        print(filename)
        stats = stats.append(pd.Series(0, index=stats.columns), ignore_index=True)
        means.append(0)
        t = threading.Thread(target=get_mean, args=(filename, means, index))
        threads.append(t)
        t.start()
        index += 1
        
        # plot(sizes)
for t in threads:
    t.join()

stats['mean'] = means
stats

kernel-MESA_SHADER_FRAGMENT_func1_main_1.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_11.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_13.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_15.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_17.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_19.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_21.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_23.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_25.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_27.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_29.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_3.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_31.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_33.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_35.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_37.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_39.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_41.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_43.traceg
kernel-MESA_SHADER_FRAGMENT_func1_main_45.traceg
kernel-MESA_SHADER_FRA

,mean
0,5.835526
1,4.457949
2,6.191448
3,4.572082
4,4.840803
5,5.263288
6,3.703431
7,21.918615
8,6.773038
9,8.081081


In [168]:
fig = go.Figure()

to_plot = stats.sort_values('mean').reset_index(drop=True)
fig.add_trace(go.Bar(x=to_plot.index, y=to_plot['mean'], name='mean'))

fig.update_layout(
        xaxis=dict(
            title="CTA",
            titlefont=dict(size=25, color="black",family='sans-serif'),
            tickfont=dict(size=20, color="black",family='sans-serif'),
            # autorange=True,
            gridcolor="gainsboro",
            # hide tick numbers but keep label
        ),
        
        yaxis=dict(
            title="# of Cache Lines",
            titlefont=dict(size=25, color="black", family='sans-serif'),
            tickfont=dict(size=20, color="black", family='sans-serif'),
            # autorange=True,
            gridcolor="gainsboro",
        ),
        width=800, height=300,
        title_font_family="sans-serif",
        title_font_size=25,
        margin=dict(l=20, r=10, t=70, b=20),
        # legend to top
        legend=dict(
            yanchor="top",
            y=1.1,
            xanchor="left",
            x=0.01,
            font=dict(size=25, family='sans-serif'),
            # orientation="h",
        ),
        plot_bgcolor="white",
    )

# hide x axis
# fig.update_xaxes(visible=False)